# Rapanden Rasmus: using GeoJSON Geometries

_Rapanden Rasmus fra Rinkenæs Sogn_ is the first line of a popular Danish song for children. It is about a duck named
Rasmus from Rinkenæs Sogn (parish).

Let's find out where it is.

We will look up its geometry as GeoJSON and map it with GeoPandas.

In [ ]:
%matplotlib inline

import requests
import pandas as pd
import matplotlib.pyplot as plt

import geopandas
import contextily

import matplotlib as mpl
import urllib
mpl.rcParams["figure.facecolor"]  = '0.95'
mpl.rcParams["figure.figsize"]  = (10,10)

## Prepare some data

Let's get the parish from the Danish Addresses Web API, DAWA.
First, we using the search API for parishes, then we get its geometry as GeoJSON
with the right `format` parameter.

See https://dawa.aws.dk/dok/api/generelt#dataformater for more information.
The specific information about the data for a _sogn_ (parish) is
available at https://dawa.aws.dk/dok/api/sogn#databeskrivelse.

In [ ]:
r = requests.get('https://dawa.aws.dk/sogne?navn=Rinken%C3%A6s')
rinkenaes_sogn = r.json()[0]
rinkenaes_sogn_url = rinkenaes_sogn['href']
print(rinkenaes_sogn_url)

In [ ]:
# We can get it like so...
r = requests.get(rinkenaes_sogn_url, params={'format':'geojson'})
rinkenaes_geo_url = r.url
rinkenaes_geo = r.json()
print(rinkenaes_geo)

In [ ]:
# ...but it is even easier to load it directly into GeoPandas from the URL
gdf = geopandas.read_file(rinkenaes_geo_url)
gdf.head()

This is what _Rinkenæs Sogn_ looks like:

In [ ]:
ax = gdf.plot(color='b', alpha=0.5)
contextily.add_basemap(ax, crs=gdf.crs.to_string(), source=contextily.providers.OpenStreetMap.Mapnik)
ax.set_title('Rinkenæs Sogn');

It would be nice to add some context, let's get its _kommune_, the municipality.

In [ ]:
r = requests.get('https://dawa.aws.dk/kommuner/reverse', params={'x': gdf.visueltcenter_x[0],
                                                                 'y': gdf.visueltcenter_y[0]})
kommune = r.json()
print(kommune['navn'], kommune['href'])

In [ ]:
kommune_geo_url = requests.Request('GET', kommune['href'], params={'format': 'geojson'}).prepare().url
kommune_gdf = geopandas.read_file(kommune_geo_url)

In [ ]:
ax = kommune_gdf.plot(alpha=.2, edgecolor='k')
gdf.plot(ax=ax, color='b', alpha=0.5)
contextily.add_basemap(ax, crs=gdf.crs.to_string(), source=contextily.providers.OpenStreetMap.Mapnik)
ax.text(gdf.visueltcenter_x[0], gdf.visueltcenter_y[0], s=gdf.navn[0])
ax.text(kommune_gdf.visueltcenter_x[0], kommune_gdf.visueltcenter_y[0], s=kommune_gdf.navn[0])
ax.set_title(f'Rinkenæs Sogn in {kommune["navn"]} municipality');